<a href="https://colab.research.google.com/github/kashperova/cv-iasa/blob/main/notebooks/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

In [2]:
!git clone https://github.com/kashperova/cv-iasa.git

Cloning into 'cv-iasa'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 45 (delta 1), reused 45 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 930.32 KiB | 2.04 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
%cd cv-iasa/src

/content/cv-iasa/src


In [3]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: skashperova (kashperova-test). Use `wandb login --relogin` to force relogin


True

In [4]:
wandb.init(
    project="cv-iasa",
    name="lab2"
)

In [ ]:
!pip install hydra-core torchmetrics

In [5]:
import hydra
from torch import optim, nn

from models import AlexNet
from modules.trainers import BaseTrainer
from utils.metrics import Task, Metrics

In [6]:
num_classes = 10
model = AlexNet(num_classes=num_classes)
metrics = Metrics(task=Task.MULTICLASS_CLASSIFICATION, average="macro", num_classes=num_classes)

In [7]:
hydra.initialize(config_path="../src/config/trainers", version_base=None)
config = hydra.compose(config_name="alexnet.yaml")

In [9]:
loss_fn = nn.CrossEntropyLoss()

In [14]:
LOSSES = []

In [15]:
from copy import deepcopy
from sklearn.model_selection import KFold
from torch.utils.data import Subset


n_splits = 3
k_fold = KFold(n_splits=n_splits, shuffle=True, random_state=config.random_seed)

for fold, (train_idx, val_idx) in enumerate(k_fold.split(train_dataset)):
    print(f"\nStarting fold {fold + 1}/{n_splits}")
    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)

    new_model = deepcopy(model)
    optimizer = optim.SGD(new_model.parameters(), lr=0.01, momentum=0.9)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=1, min_lr=1e-6)

    trainer = BaseTrainer(
        model=model,
        train_dataset=train_subset,
        eval_dataset=val_subset,
        config=config,
        loss_fn=loss_fn,
        optimizer=optimizer,
        lr_scheduler=lr_scheduler,
        metrics=metrics,
        save_dir=f"/content/"
    )
    new_model = new_model.to(trainer.device)
    trainer.model = new_model

    trainer.train(verbose=True)
    LOSSES.append((trainer.train_losses, trainer.eval_losses))


Starting fold 1/3


Training:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch [1/5], Train Loss: 0.2219, Valid Loss: 0.0645

Train metrics: 
Accuracy: 0.9326
Recall: 0.9321
Precision: 0.9325
F1: 0.9322


Valid metrics: 
Accuracy: 0.9814
Recall: 0.9809
Precision: 0.9818
F1: 0.9812



Training:  20%|██        | 1/5 [02:30<10:01, 150.35s/it]

Epoch [2/5], Train Loss: 0.0656, Valid Loss: 0.0564

Train metrics: 
Accuracy: 0.9812
Recall: 0.9811
Precision: 0.9811
F1: 0.9811


Valid metrics: 
Accuracy: 0.9863
Recall: 0.9862
Precision: 0.9864
F1: 0.9862



...

Epoch [3/5], Train Loss: 0.0509, Valid Loss: 0.0446

Train metrics: 
Accuracy: 0.9865
Recall: 0.9864
Precision: 0.9865
F1: 0.9864


Valid metrics: 
Accuracy: 0.9893
Recall: 0.9891
Precision: 0.9893
F1: 0.9892



Training:  60%|██████    | 3/5 [07:28<04:58, 149.32s/it]

Epoch [4/5], Train Loss: 0.0421, Valid Loss: 0.0406

Train metrics: 
Accuracy: 0.9887
Recall: 0.9886
Precision: 0.9886
F1: 0.9886


Valid metrics: 
Accuracy: 0.9897
Recall: 0.9897
Precision: 0.9898
F1: 0.9897



Training: 100%|██████████| 5/5 [12:29<00:00, 149.81s/it]

Epoch [5/5], Train Loss: 0.0361, Valid Loss: 0.0415

Train metrics: 
Accuracy: 0.9918
Recall: 0.9917
Precision: 0.9918
F1: 0.9917


Valid metrics: 
Accuracy: 0.9913
Recall: 0.9912
Precision: 0.9914
F1: 0.9912



In [16]:
from utils.plots import plot_losses


plot_losses(LOSSES[0][0], LOSSES[0][1])

In [17]:
from utils.plots import plot_losses


plot_losses(LOSSES[1][0], LOSSES[1][1])

In [18]:
from utils.plots import plot_losses


plot_losses(LOSSES[2][0], LOSSES[2][1])